<a href="https://colab.research.google.com/github.com/jasper166/recommendation/edit/main/pre_process_yoochoose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

path = '/content/drive/MyDrive/KDL&KPDL'
os.chdir(path)
os.listdir()

['yoochoose-data', 'SBR-clone']

In [ ]:
os.listdir('yoochoose-data')

In [ ]:
import os
import pandas as pd
from datetime import datetime
import time

filepath = 'yoochoose-data/yoochoose-clicks.dat'
# os.getcwd('input')
dataset = pd.read_csv(filepath, names = ['SessionId', 'DateTime', 'ItemId', 'Category'])


<ipython-input-3-e81dbfc06bfe>:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(filepath, names = ['SessionId', 'DateTime', 'ItemId', 'Category'])


In [ ]:
dataset['DateTime'] = dataset['DateTime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
dataset['Timestamp'] = dataset['DateTime'].apply(lambda x: x.strftime('%s.%f'))
dataset = dataset.sort_values(by = ['SessionId', 'DateTime'], ascending = True)
print(dataset.shape)
dataset.head()

thống kê các thuộc tính trong dataset

In [ ]:
print("length of dataset: ", len(dataset))
print("number of distinct item: ", dataset['ItemId'].unique().shape[0])
print("number of distinct session: ", dataset['SessionId'].unique().shape[0])
print("number of distinct time: ", dataset['DateTime'].unique().shape[0])

check Category

In [ ]:
#category
category_dict = {}
for i in dataset['Category']:
  if i not in category_dict:
    category_dict[i] = 1
    continue
  category_dict[i] += 1
category_dict = sorted(category_dict, key = lambda x: x[1], reverse = True)
print(category_dict[:32])

In [ ]:
keys = [i[0] for i in category_dict[:32]]
for j,i in enumerate(keys):
  if i == 0: i = -2
  elif i == 'S': i = -1
  else: i = int(i)
  keys[j] = i
cnt = 13
for j,i in enumerate(keys):
  if i > 12:
    keys[j] = cnt
    cnt+=1
keys


In [ ]:
values = [i[1] for i in category_dict[:32]]
plt.bar(keys, values)
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Bar Chart of Dictionary')
plt.show()

Thống kê số lượng xuất hiện của các item

In [ ]:
#Thống kê số lần xuất hiện của các item --> data_dict = {'itemId1': count1, 'itemId2': count2,...}
data_dict = {}
for i in dataset['ItemId']:
  if i not in data_dict:
    data_dict[i] = 1
    continue
  data_dict[i] += 1
data_dict = dict(sorted(data_dict.items(), key=lambda x: x[1]))
data_dict

In [ ]:
#thống kê số item xuất hiện n lần với n chạy từ 1 đến số lần xuất hiện tối đa của 1 item
# --> num_dict = {1: numofItems, 2: numofItems, ...}
num_dict = {}
for i in data_dict:
  if data_dict[i] not in num_dict:
    num_dict[data_dict[i]] =1
    continue
  num_dict[data_dict[i]] += 1
num_dict

In [ ]:
#Cộng dồn từ phần tử thứ 9 trở đi để vẽ biểu đồ
new_dict = {}
for i in range(1,9):
  new_dict[i] = num_dict[i]
sum = 0
for i in range(9, 147420):
  if i in num_dict.keys():
    sum += num_dict[i]
new_dict[9] = sum

# Tạo danh sách các nhãn và phần trăm tương ứng
labels = [str(key) for key in new_dict.keys()]
sizes = [value / total * 100 for value in new_dict.values()]
labels[8] = '>=9'
# Vẽ biểu đồ pie
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Đảm bảo biểu đồ có hình dạng tròn
plt.title('Pie Chart of item occurence')
plt.show()

**<h1/>Bắt đầu tiền xử lý dữ liệu**

Bên dưới ta sẽ lọc bỏ các itemId có ít hơn 5 lượt xuất hiện

In [ ]:
# Thống kê số lượt xuất hiện và lọc ra các ItemId có trên 5 lượt xuất hiện
df_item_count = dataset[['ItemId', 'SessionId']].groupby('ItemId').count().sort_values(by = 'SessionId', ascending = False)
df_item_count.columns = ['CountItemId']
df_item_count_5 = df_item_count[df_item_count['CountItemId'] < 5]
# Lọc khỏi dataset những ItemId có ít hơn 5 lượt xuất hiện
dataset = dataset[~dataset['ItemId'].isin(list(df_item_count_5.index))]
print(dataset.shape)

Loại bỏ các Session độ dài 1

In [ ]:
session_length = dataset.groupby('SessionId').size()

# Lọc ra các SessionId có độ dài lớn hơn 1
valid_sessions = session_length[session_length > 1].index

# Lọc dataset với các SessionId có độ dài lớn hơn 1
dataset = dataset[dataset['SessionId'].isin(valid_sessions)]

print(dataset.shape)

In [ ]:
from datetime import timedelta
# Phân chia tập train/test sao cho tập test là 1 ngày gần đây nhất và train là dữ liệu còn lại
maxdate = dataset['DateTime'].max()
mindate = maxdate - timedelta(days = 1)
test = dataset[dataset['DateTime'] >= mindate]
dataset = dataset[dataset['DateTime'] <= mindate]
print(dataset.shape)
print(test.shape)

In [ ]:
import numpy as np

# list các sessionIds
sessIds = list(dataset['SessionId'].unique())
# Lấy ngẫu nhiên 1/64 số lượng các session
n_filter = int(len(sessIds)/64)
np.random.shuffle(sessIds)
sessIdsFilter = sessIds[:n_filter]
# Lựa chọn các 1/64 session làm dataset train (dữ liệu này bao gồm cả train và validation)
# Set index là sessionId để filter nhanh hơn
dataset.set_index('SessionId', inplace=True)
dataset_filter = dataset[dataset.index.isin(sessIdsFilter)]
dataset_filter = dataset_filter.reset_index()
print(dataset_filter.shape)

In [ ]:
train_sess = dataset_filter[['SessionId', 'ItemId', 'Timestamp']].groupby('SessionId').apply(lambda x: dict(zip(x['ItemId'], x['Timestamp'])))
test_sess = test[['SessionId', 'ItemId', 'Timestamp']].groupby('SessionId').apply(lambda x: dict(zip(x['ItemId'], x['Timestamp'])))
train_sess[:5]

In [ ]:
sessDict = {214834865: '1396808691.295000', 214706441: '1396808691.426000', 214820225: '1396808691.422000'}

def _preprocess_sess_dict(sessDict):
    sessDictTime = dict([(v, k) for (k, v) in sessDict.items()])
    sessSort = sorted(sessDictTime.items(), reverse = False)
    times = [item[0] for item in sessSort]
    itemIds = [item[1] for item in sessSort]
    inp_seq = []
    labels = []
    inp_time = []

    for i in range(len(sessSort)):
        if i >= 1:
            inp_seq += [itemIds[:i]]
            labels += [itemIds[i]]
            inp_time += [times[i]]
    return inp_seq, inp_time, labels, itemIds

#example for sessDict
inp_seq, inp_time, labels, itemIds = _preprocess_sess_dict(sessDict)
print('input sequences: ', inp_seq)
print('input times: ', inp_time)
print('targets: ', labels)
print('sequence: ', itemIds)

In [ ]:
def _preprocess_data(data_sess):
    inp_seqs = []
    inp_times = []
    labels = []
    sequences = []
    sessIds = list(data_sess.index)
    for sessId in sessIds:
        sessDict = data_sess.loc[sessId]
        inp_seq, inp_time, label, sequence = _preprocess_sess_dict(sessDict)
        inp_seqs += inp_seq
        inp_times += inp_time
        labels += label
        sequences += sequence
    return inp_seqs, inp_times, labels, sequences

train_inp_seqs, train_inp_dates, train_labs, train_sequences = _preprocess_data(train_sess)
test_inp_seqs, test_inp_dates, test_labs, test_sequences = _preprocess_data(test_sess)

train = (train_inp_seqs, train_labs)
test = (test_inp_seqs, test_labs)

print('Done.')
for i in range(4):
  print("input is: ", train_inp_seqs[i], "with output: ", train_labs[i])

In [ ]:
# Các token default
PAD_token = 0  # token padding cho câu ngắn

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.item2index = {}
        self.item2count = {}
        self.index2item = {PAD_token: "PAD"}
        self.num_items = 1  # số lượng mặc định ban đầu là 1 ứng với PAD_token
# sessDict = [ 21483,21483, 21893, 21893,83390,903wu9i084,9023893,8389]
# addSequence(sessDict)
# addItem(214834865)
# addItem(214706441)
# addItem(214820225)
# addItem(214834856)
    def addSenquence(self, data):
        for sequence in data:
          for item in sequence:
              self.addItem(item)

    # Thêm một item vào hệ thống
    def addItem(self, item):
        if item not in self.item2index:
            self.item2index[item] = self.num_items #item2index[214834865] = 1, ....
            self.item2count[item] = 1 #count[214...] = 1
            self.index2item[self.num_items] = item #index2item[1] = 214....
            self.num_items += 1 #num_items = 2
        else:
            self.item2count[item] += 1



    # Hàm convert sequence (1 session)  về chuỗi các indices
    def _seqItem2seqIndex(self, x):
        return [voc.item2index[item] if item in voc.item2index else 0 for item in x]

In [ ]:
from itertools import chain
seq_targets = train[0] + [train[1]] + test[0] + [test[1]] #tat ca cac itemid da tung xuat hien (co lap)
sessionIds = list(chain.from_iterable(seq_targets))
sessionIds = set(sessionIds) # ->> 1 set ko con phan tu lap nua
print('Number of sessionIds: ', len(sessionIds))

In [ ]:
voc = Voc('DictItemId')
voc.addSenquence(seq_targets)

# Convert thử nghiệm một sequence itemIds
print('sequence of itemIds: ', train[0][7])
print('converted indices: ', voc._seqItem2seqIndex(train[0][7]))

In [ ]:
train_x_index = [voc._seqItem2seqIndex(seq) for seq in train[0]] #input train
test_x_index = [voc._seqItem2seqIndex(seq) for seq in test[0]] #input test
train_y_index = voc._seqItem2seqIndex(train[1]) #output train
test_y_index = voc._seqItem2seqIndex(test[1])#output test
train_index = (train_x_index, train_y_index)
test_index = (test_x_index, test_y_index)
print(voc.num_items)

In [ ]:
import pickle
import os

def _save_file(filename, obj):
  with open(filename, 'wb') as fn:
    pickle.dump(obj, fn)

# Tạo folder yoochoose-data-64 để lưu dữ liệu train/test nếu chưa tồn tại
if not os.path.exists('yoochoose-data/yoochoose-data-64'):
  os.mkdir('yoochoose-data/yoochoose-data-64')

# Lưu train/test
_save_file('yoochoose-data/yoochoose-data-64/train.pkl', train_index)
_save_file('yoochoose-data/yoochoose-data-64/test.pkl', test_index)

